In [193]:
#Import library
import pandas as pd
import numpy as np
from datetime import datetime
from statistics import mode
from collections import Counter #untuk mencari modus dan untuk menghindari bimodus

In [194]:
data = pd.read_csv('order_brush_order.csv')

In [195]:
data = data.sort_values(['shopid', 'event_time'])

In [196]:
data.head()

,orderid,shopid,userid,event_time
59,31086409141107,10009,196962305,2019-12-27 03:06:50
76187,31144571933461,10051,2854032,2019-12-27 19:16:11
9055,31254979546679,10051,48600461,2019-12-29 01:56:19
188513,31195675919209,10061,168750452,2019-12-28 09:27:55
174620,31205132327893,10061,194819216,2019-12-28 12:05:32


In [197]:
shops = data.groupby(by='shopid').count()

In [198]:
susp1 =  list(shops[shops['orderid'] == 3].index)

In [199]:
#Mencari brush user untuk shop dengan 3 transaksi
susps = []  #shopid yang melakukan brushing
suspu = []  #userid yang melakukan brushing
for i in susp1:
    dt = data[data['shopid']==i]
    time0 = datetime.strptime(dt.iloc[0,3], '%Y-%m-%d %H:%M:%S')
    time1 = datetime.strptime(dt.iloc[-1,3], '%Y-%m-%d %H:%M:%S')
    if len(set(dt['userid'])) == 1 and ((time1-time0).total_seconds() <= 3600):
        susps.append(dt.iloc[0,1])
        suspu.append(dt.iloc[0,2])

In [200]:
#Membuat data frame untuk jawaban dengan nilai awal semua userid = 0
csv = pd.DataFrame({'shopid' : shops.index, 'userid' : [0 for i in range(len(shops))]})

In [201]:
#Menambahkan userid yang telah terdeteksi brushing ke dataframe jawaban
for i,j in zip(susps, suspu):
    csv.iloc[csv[csv['shopid'] == i].index.values[0],1] = j

In [202]:
csv[csv['shopid'].isin(susps)]

,shopid,userid
599,3124091,214568881
3133,50682734,214365114
7687,129113839,215408773
9645,153444897,169948888
9671,153672697,214778616
9795,155016230,188431260
10108,157939195,69563623
10598,161196859,180862522
11716,163351089,214778616
16892,201428849,89254393


In [203]:
susp2 = list(shops[shops['orderid'] > 3].index)

In [204]:
#Membuat dataframe baru dengan jumlah transaksi lebih dari 3
data2 = data[data['shopid'].isin(susp2)].sort_values(['shopid','event_time'])

In [205]:
#Merubah kolom event_time menjadi format waktu
data2['event_time'] = data2['event_time'].astype('datetime64[ns]')

In [206]:
data2

,orderid,shopid,userid,event_time
188513,31195675919209,10061,168750452,2019-12-28 09:27:55
174620,31205132327893,10061,194819216,2019-12-28 12:05:32
200439,31412115824794,10061,130633421,2019-12-30 21:35:15
145426,31431527100615,10061,62464559,2019-12-31 02:58:48
167859,31075686185309,10084,4401933,2019-12-27 00:08:06
...,...,...,...,...
182444,31407699385160,213900783,215251024,2019-12-30 20:21:39
108246,31468414075366,214432425,1134243,2019-12-31 13:13:34
133194,31505523761333,214432425,1134243,2019-12-31 23:32:03
83893,31506349293329,214432425,52594422,2019-12-31 23:45:49


In [207]:
#Fungsi untuk mencari userid yang brushing dengan input dataframe dengan 1 shopid
def getuser(test):
    modus = True
    sub = []
    for i in range((len(test))):
        time0 = test.iloc[i,3]
        usr = 1
        usrn = [test.iloc[i,2]]
        for j in range(i+1, len(test)):
            if (test.iloc[j,3] - time0).total_seconds() <= 3600:
                usr+=1
                usrn.append(test.iloc[j,2])
            else:
                break
        for k in set(usrn):
            if usrn.count(k) >= 3:
                if k not in sub:
                    sub.append(k)
        if len(usrn) > 0:
            if usr/len(set(usrn)) >= 3:
                try:
                    mode(usrn)
                except:
                    modus = False
                    usrn = Counter(usrn)
                    sub.extend([usrn.most_common()[i][0] for i in range(2)])
                if modus:
                    sub.append(mode(usrn))     
        modus = True
    if len(sub) == 0 or len(set(sub)) > 2: 
        return 0
    else:
        sub = list(set(sub))
        sub.sort()
        return(sub)

In [208]:
#Mengaplikasikan fungsi getuser ke dataframe soal
def fuck(up):
    ids = [] #shopid yang brushing
    idu = [] #userid yang brushing
    for i in (up):
        test = data2[data2['shopid'] == i].reset_index(drop=True)
        ans = getuser(test)
        if bool(ans):
            ids.append(i)
            idu.append(ans)
    return [ids, idu]

In [209]:
%%time
gd = fuck(susp2)

CPU times: user 3min 6s, sys: 51.9 ms, total: 3min 6s
Wall time: 3min 6s


In [210]:
len(gd[0])

371

In [211]:
#Membuat dataframe baru untuk jawaban yang baru ditemukan
csv2 = pd.DataFrame({'shopid': gd[0], 'userid' : gd[1]}) 

In [212]:
#Membuat userid menjadi string dan menambahkan &
csv2['userid'] = csv2['userid'].apply(lambda x: '&'.join([str(i) for i in x]))

In [213]:
#Menggabungkan jawaban baru dengan jawaban awal
csv3 = csv.append(csv2, ignore_index=True)

In [214]:
#Mengahpus duplikat setelah digabungkan
csv3 = csv3.drop_duplicates(subset='shopid', keep='last')

In [215]:
csv3[csv3['userid'] != 0]

,shopid,userid
599,3124091,214568881
3133,50682734,214365114
7687,129113839,215408773
9645,153444897,169948888
9671,153672697,214778616
...,...,...
19136,201557550,214605778
19137,203531250,114282846
19138,204225676,198662175
19139,208696908,214111334


In [216]:
csv3.to_csv('uji.csv', index =False)

In [217]:
len(data)

222750